In [18]:
import pickle
import numpy as np
import pandas as pd
import zipfile
from collections import defaultdict
from surprise import Dataset, SVD, Reader
from surprise.model_selection import cross_validate

In [19]:
# Source: Modified from https://github.com/NicolasHug/Surprise/blob/master/examples/top_n_recommendations.py
def get_top_n_for_user(predictions, user_id, n=10):
    """
    Return the top-N recommendations for a given user from a set of predictions.

    Args:
        predictions (list of Prediction objects): The list of predictions, as returned by the test method of an algorithm.
        user_id (str or int): The user ID for whom to get recommendations.
        n (int): The number of recommendations to return. Default is 10.

    Returns:
        list or str: A sorted list of tuples [(item_id, estimated_rating), ...] of size n for the given user_id,
                     or a message if the user_id is not found.
    """

    # Filter predictions for the given user_id
    user_predictions = [(iid, est) for uid, iid, true_r, est, _ in predictions if uid == user_id]

    # If user_id is not found, return a message
    if not user_predictions:
        return f"User ID {user_id} not found in predictions."

    # Sort by estimated rating in descending order
    user_predictions.sort(key=lambda x: x[1], reverse=True)

    # Return top-N recommendations
    return user_predictions[:n]

In [20]:
# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_builtin("ml-100k")
trainset = data.build_full_trainset()
links = pd.read_csv("data/links.csv")

In [21]:
# Get the raw training data
raw_trainset = data.raw_ratings

# Convert training data to use TMDB IDs
ratings_data = []
for uid, mid, rating, timestamp in raw_trainset:
    # Convert movie ID to int for matching with links DataFrame
    movie_id = int(mid)
    # Get the TMDB ID from links DataFrame
    tmdb_id = links[links['movieId'] == movie_id]['tmdbId'].values
    # Only add rating if TMDB ID exists
    if len(tmdb_id) > 0:
        ratings_data.append({
            'userID': str(uid),
            'movieID': str(tmdb_id[0]),
            'rating': rating
        })

# Convert to DataFrame
df = pd.DataFrame(ratings_data)

# Create a new Reader object
reader = Reader(rating_scale=(1, 5))

# Create a new Dataset object with TMDB IDs
data = Dataset.load_from_df(df[['userID', 'movieID', 'rating']], reader)

# Build the full trainset
trainset = data.build_full_trainset()

In [22]:

algo = SVD()
algo.fit(trainset)
pd.DataFrame(cross_validate(algo, data, measures=["RMSE", "MAE"], cv=3, verbose=True))

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9431  0.9430  0.9488  0.9450  0.0027  
MAE (testset)     0.7442  0.7498  0.7475  0.7472  0.0023  
Fit time          0.21    0.22    0.26    0.23    0.02    
Test time         0.05    0.05    0.05    0.05    0.00    


,test_rmse,test_mae,fit_time,test_time
0,0.943090,0.744238,0.213535,0.047904
1,0.943019,0.749758,0.218082,0.045822
2,0.948809,0.747478,0.261923,0.045748


In [23]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [24]:
get_top_n_for_user(predictions, uid, n=10)


[('19760.0', 5),
 ('22588.0', 4.989641945767773),
 ('9073.0', 4.956996495431574),
 ('18215.0', 4.906787174107573),
 ('32631.0', 4.8828396677712345),
 ('18069.0', 4.878269811933069),
 ('15730.0', 4.872297848963366),
 ('2292.0', 4.871217287155522),
 ('712.0', 4.84605830354883),
 ('11318.0', 4.841064759645194)]

In [25]:
with open("../backend/model.pkl", "wb") as file:
    pickle.dump(predictions, file)

In [27]:
with zipfile.ZipFile('../backend/model.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('../backend/model.pkl', 'model.pkl')